In [ ]:

# ETHNOS

# Required imports
from selenium import webdriver
# from selenium.webdriver.common.keys import Keys  # Not used
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os
from webdriver_manager.chrome import ChromeDriverManager
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.firefox.service import Service as FirefoxService
from webdriver_manager.firefox import GeckoDriverManager
from selenium.common.exceptions import TimeoutException

# User inputs for the URL and tag name
url = input('Enter please the url of the main page ')
tname = input('print the tag name')

# File naming setup
a = 'Ethnos'
b = '_Wdate'
csv = '.csv'
simple = a + tname + csv
wd = a + tname + b + csv

# Initialize lists and stopping date
date_list = []
tag_list = []
stop_date = 20241201

# Initialize Firefox WebDriver
driver = webdriver.Firefox(service=FirefoxService(GeckoDriverManager().install()))
driver.get(url)
time.sleep(2)
driver.maximize_window()
time.sleep(2)

# Handle cookies pop-up by clicking 'Disagree'
disagree = WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div/div/div[2]/div/button[2]"))).click()

# Scroll to the "Load more" button initially to make it visible
load_button = driver.find_element(By.CLASS_NAME, "read-more-posts")
driver.execute_script("arguments[0].scrollIntoView(false);", load_button)
time.sleep(1.5)

# Function to dynamically update the XPath for the "Load more" button
xpath = '/html/body/div[2]/main/div[2]/div[1]/div[1]/div[1]/div/div[12]/button'
def new_button_xpath():
    global xpath
    digits = xpath[59:63]
    while digits.isdigit() == False:
        digits = digits[:-1]
    nr_of_digits = len(digits)
    digits = int(digits) + 9
    digits = str(digits)
    xpath = xpath[:59] + digits + xpath[59 + nr_of_digits:]

# Function that checks the date of the last article on the page to determine whether to stop
def scrolling_date_check(driver, stop_date):
    global flag
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'lxml')
    suggested_articles = soup.find('div', class_='main-content')
    date_check = suggested_articles.find_all('div', class_='single-post-container')[-1]
    date_check = date_check.find('div', class_='single-category')
    article_time = date_check.find_all("a")[1].get_text()
    new_date = article_time[:-6].split('.')
    new_date.reverse()
    for i in range(len(new_date)):
        if '\n' in new_date[i]:
            new_date[i] = new_date[i][-2:]
    new_date = "".join(new_date)
    try:
        new_date = int(new_date)
    except ValueError:
        new_date = article_time[:-6].split('/')
        new_date.reverse()
        new_date = int("".join(new_date))
    if new_date < stop_date:
        print('we reached the date limit')
        flag = False

# Function to collect tags and dates from visible articles
def collecting_fun(driver):
    global tag_list
    global date_list

    def tag_function(inside):
        global tag_list
        page = requests.get(inside)
        soup = BeautifulSoup(page.content, 'html.parser')
        lifestyle = soup.find_all('div', class_='category')
        try:
            if len(lifestyle) > 1:
                print('vrika xalasmeno arhtro. Lifestyle Magazine')
                print('its url is ', inside)
                print('Please check the url i gave you..')
            else:
                tag_section = soup.find('ul', class_='list-inline tags-list')
                b = tag_section.find_all("a")
                inside_tag_list = []
                for a in b:
                    tag = a.get_text()
                    inside_tag_list.append(tag)
                tag_list.append(inside_tag_list)
        except:
            print('vrika xalasmeno arthro')
            print('Please check for broken url: ', inside)

    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'lxml')
    suggested_articles = soup.find('div', class_='main-content')

    # Iterate through articles
    for h in suggested_articles('div', class_='single-post-container'):
        date_check = h.find('a')
        date_check = h.find('div', class_='single-category')
        single_category = date_check.find_all('a')
        article_time = single_category[1].get_text()
        new_date = article_time[:-6].split('.')
        new_date.reverse()
        for i in range(len(new_date)):
            if '\n' in new_date[i]:
                new_date[i] = new_date[i][-2:]
        new_date = "".join(new_date)
        try:
            new_date = int(new_date)
        except ValueError:
            new_date = article_time[:-6].split('/')
            new_date.reverse()
            new_date = int("".join(new_date))
        if new_date < stop_date:
            print('I have already found the last article man!!')
            break
        else:
            inside_tag_list = []
            date_list.append(new_date)
            outer_tag = single_category[0].get_text()
            inside_tag_list.append(outer_tag)
            article_url = h.find('a').get('href')
            https = "https://www.ethnos.gr"
            if https not in article_url:
                article_url = https + article_url
            else:
                continue
            tag_function(article_url)
            time.sleep(0.3)

# Scrolling loop: keeps clicking "Load more" and checks dates
nr_of_button_pressed_times = 2
nr_of_button_pressed_count = nr_of_button_pressed_times
flag = True
nr = 0  # number of scrolls

while flag:
    load_button = driver.find_element(By.XPATH, xpath)
    try:
        driver.execute_script("arguments[0].scrollIntoView(false);", load_button)
        time.sleep(1.5)
        load_more = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, xpath))).click()
    except TimeoutException:
        print('load button cannot be found. Probably we reached the end of scrolling. \n No more articles appear')
        flag = False
    time.sleep(1)
    nr_of_button_pressed_count -= 1
    if nr_of_button_pressed_count == 0:
        scrolling_date_check(driver, stop_date)
        nr_of_button_pressed_count = nr_of_button_pressed_times
    nr += 1
    new_button_xpath()
    time.sleep(1)

print('nr is ', nr)

# After scrolling is done, collect all tags and dates
collecting_fun(driver)

# Processing tag list and date list into DataFrames and exporting to CSV
import pandas as pd

# Flatten tag list into a single list for counting
new_tag_list = []
for i in range(len(tag_list)):
    for j in range(len(tag_list[i])):
        new_tag_list.append(tag_list[i][j])

# Count frequency of each tag
res = {}
for i in new_tag_list:
    res[i] = new_tag_list.count(i)

# Convert the tag-frequency dictionary to a list for DataFrame
resultList = list(map(list, res.items()))
df_costliness = pd.DataFrame(resultList)
df_costliness.to_csv(simple, header=False)

# Convert int date format (yyyymmdd) to dotted format (dd.mm.yyyy)
ddate_list = date_list
ttag_list = tag_list
ndlist = []
for i in ddate_list:
    i = str(i)
    y = i[:4]
    m = i[4:6]
    d = i[6:]
    ddate = d + '.' + m + '.' + y
    ndlist.append(ddate)

print(ddate_list)
print(ndlist)

# Connect each tag with its corresponding article date
n = 0
date_con_list = []
for i in ttag_list:
    n += 1
    for tag in i:
        inside = []
        inside.append(tag)
        inside.append(ddate_list[n - 1])
        date_con_list.append(inside)

print(date_con_list)

df_connected = pd.DataFrame(date_con_list)
df_connected.to_csv(wd, header=False)

# Final output logs
print(len(tag_list))
print(len(date_list))
print(tag_list)
print('the last date collected is ', date_list[-1])
print('the nr of the tags mentioned is ', len(resultList))


we reached the date limit
nr is  2
I have already found the last article man!!
[20250605, 20250507, 20250502, 20250430, 20250412, 20250408, 20250328, 20250325, 20250316, 20250314, 20250310, 20250221, 20250220, 20250124, 20250120, 20250113, 20250110, 20241223, 20241221, 20241218]
['05.06.2025', '07.05.2025', '02.05.2025', '30.04.2025', '12.04.2025', '08.04.2025', '28.03.2025', '25.03.2025', '16.03.2025', '14.03.2025', '10.03.2025', '21.02.2025', '20.02.2025', '24.01.2025', '20.01.2025', '13.01.2025', '10.01.2025', '23.12.2024', '21.12.2024', '18.12.2024']
[['κυβέρνηση', 20250605], ['δημοσκοπήσεις', 20250605], ['ειδήσεις', 20250605], ['ειδήσεις τώρα', 20250605], ['ΠΑΣΟΚ', 20250507], ['ειδήσεις', 20250507], ['Πλεύση Ελευθερίας', 20250507], ['ειδήσεις τώρα', 20250507], ['δημοσκοπήσεις', 20250507], ['δημοσκοπήσεις', 20250502], ['Κυριάκος Μητσοτάκης', 20250502], ['Νέα Δημοκρατία', 20250502], ['ειδήσεις τώρα', 20250502], ['Κυριάκος Μητσοτάκης', 20250430], ['δημοσκοπήσεις', 20250430], ['δημοσκ